- GPT - 2
  - Transformer 디코더 블럭을 여러개 쌓아서 만들었음
    - 1.셀프어텐션 레이어
      - 입력한 문장에서 단어들 간의 관계를 파악해 중요한 단어에 집중하도록 고려
    - 2. Feed Forward Neural Network
      - 각 단어 벡터를 독립적으로 변환해서 테스트의 특징을 추출
    - 3. Residual Connection & Layer Normalization
      - 각 레이어의 입력과 출력을 합산해서 안정성은 높이고 학습이 빠름
  - Self-Attention
    - Query @ key --> 어텐션 스코어 * 벨류 --> 가중합 - >softmax 확률분포
  - Positional Encoding
    - Recurrent 구조가 아님.. 단어의 순서를 구하기위서 위치정보사용(사인 코사인 정보를 번갈아가면서 사용해서)
  - Masked Self-Attention
    - 미래정보를 사용하지 않기위해서 이전단어의 정보만 사용
  - Auto-regressive 구조
    - 이전 단어를 기반으로 해서 다음 단어를 예측하는 문장을 생성
  - 레이어 : Residual, normalization

- 전체적인 동작
  - 입력 문장을 토큰화해서 임베딩 벡터로 변환
  - Postional Encoding 을 더해서 위치정보를 포함한상태 각 디코더 블럭에 전달
  - 각 디코더 블럭의 self-attention 레이어에서 단어 간의 관계를 계산
  - attention output -> Feed-Forward Network 변환됨
  - 이 과정을 디코더 블럭에서 반복 최종출력은 각 디코더가 출력한 확률에서 가장 높은 단어를 출력



# 책에있는 코드가 아닌 일반적인 방법

In [2]:
!pip install gdown

  Using cached gdown-5.2.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.66.6-py3-none-any.whl.metadata (57 kB)
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
Using cached gdown-5.2.0-py3-none-any.whl (18 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached tqdm-4.66.6-py3-none-any.whl (78 kB)
Using cached certifi-2024.8.30-py3-none-any.whl (167 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
Using cached soupsieve-2.6-py3-none-any.whl (36 kB)
Using cached urllib3-2.2.3-py3-none-any.whl (126 kB

In [3]:
# 데이터 로드
import gdown
file_id = '1EJdu56msbDuR1Awd4w9nnutIWCsaeAey'
download_url = f'https://drive.google.com/uc?id={file_id}'
gdown.download(download_url, 'fineTune_data.txt', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1EJdu56msbDuR1Awd4w9nnutIWCsaeAey
To: c:\Github\competition\factory\fineTune_data.txt
100%|██████████| 24.6k/24.6k [00:00<00:00, 3.03MB/s]


'fineTune_data.txt'

In [20]:
!pip install transformers

In [24]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
bos_token='</s>', eos_token='</s>', unk_token='<unk>',
pad_token='<pad>', mask_token='<mask>')
tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


['▁안녕',
 '하',
 '세',
 '요.',
 '▁한국어',
 '▁G',
 'P',
 'T',
 '-2',
 '▁입',
 '니다.',
 '😤',
 ':)',
 'l^o']

In [28]:
import torch
import torch_directml
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, AdamW
from transformers import get_linear_schedule_with_warmup

# 모델과 토크나이저 로드
MODEL_NAME = 'skt/kogpt2-base-v2'
tokenizer = GPT2TokenizerFast.from_pretrained(MODEL_NAME)  #Hugging Face 불러옴

# 패딩 토큰 설정
tokenizer.add_special_tokens({'pad_token': '[PAD]'}) # 시퀀스를 동일한 길이로 맞추기 위해서 사용

model = GPT2LMHeadModel.from_pretrained(MODEL_NAME) # 사전학습된 모델로드(전이학습)
model.resize_token_embeddings(len(tokenizer)) # 패딩토큰 추가했으니. 임배딩층 재 조정

# 데이터셋 클래스 정의
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length):
        self.input_ids = []
        for text in texts:
            encodings_dict = tokenizer(text, truncation=True, max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.input_ids[idx]

# 텍스트 데이터를 로드하고 전처리
with open('./fineTune_data.txt', 'r', encoding='utf-8') as f:
    texts = [line.strip() for line in f if line.strip()]

# 데이터셋 및 데이터로더 설정
dataset = TextDataset(texts, tokenizer, max_length=128)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# 학습 설정
device = torch_directml.device() if torch_directml.is_available() else torch.device('cpu')
print(device)
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)
total_steps = len(dataloader) * 3  # 3 epochs
# 초기에는 낮은학습률로 시작해서 점점 학습률을 높이는 방식
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

privateuseone:0


c:\Users\Playdata\AppData\Local\anaconda3\envs\tcml\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [29]:
from tqdm import tqdm
# 파인튜닝 루프
model.train()
for epoch in range(1):
  iterator = tqdm(dataloader)
  for batch in iterator:
      inputs, labels = batch
      inputs = inputs.to(device)
      labels = labels.to(device)

      model.zero_grad()

      outputs = model(inputs, labels=labels)
      loss = outputs.loss
      loss.backward()

      optimizer.step()
      scheduler.step()
      iterator.set_description(f"epoch:{epoch+1} Loss: {loss.item():.3f}")

# 파인튜닝 된 모델 저장
model.save_pretrained('fine_tuned_kogpt2')
tokenizer.save_pretrained('fine_tuned_kogpt2')

print("모델 파인튜닝이 완료되었습니다.")

epoch:1 Loss: 9.423: 100%|██████████| 36/36 [01:36<00:00,  2.67s/it] 


NotImplementedError: Cannot access storage of OpaqueTensorImpl

In [ ]:
texts[1]

'만일 김첨지가 주기를 띠지 않았던들 한 발을 대문에 들여놓았을 제 그곳을 지배하는 무시무시한 정적(靜寂) ― 폭풍우가 지나간 뒤의 바다 같은 정적이 다리가 떨렸으리라.'

In [ ]:
# 파인튜닝된 모델과 토크나이져 로드
model = GPT2LMHeadModel.from_pretrained('fine_tuned_kogpt2')
tokenizer = GPT2TokenizerFast.from_pretrained('fine_tuned_kogpt2')

In [ ]:
# 입력할 문장
input_text = '피부조직이 '
# 토큰화
input_ids = tokenizer.encode(input_text,return_tensors='pt')
# gpu or cpu 모델을 전송
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
input_ids = input_ids.to(device)
# 텍스트 생성 - 역전파가 필요없으므로 no_grad
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      max_length=128,
      num_beams=5,
      repetition_penalty=2.0,
      no_repeat_ngram_size=2,
      temperature=0.7,
      top_k = 50,
      top_p = 0.95
  )
  # 토큰을 텍스트로 디코딩
generated = tokenizer.decode(generated_ids[0],skip_special_tokens=True)
# 결과 출력
print(generated)

피부조직이 뻣뻣하게 굳는 것을 막아주는 역할을 한다.
또한 혈액순환을 원활하게 해주며 신진대사를 촉진시켜 혈중 콜레스테롤 수치를 낮춰준다.
이러한 효과를 볼 수 있는 방법은 크게 두 가지로 나눌 수 있다.
하나는 비타민C를 섭취하는 것이다.
비타민C는 우리 몸에 필요한 필수 영양소 중 하나이기 때문에 하루 한 알씩 꾸준히 섭취하면 좋다.
또 다른 하나는 비타민D가 풍부한 음식을 먹는 것이다.
그렇다면 비타민 D는 어떻게 섭취해야 하는 것일까.
우선 비타민 C는 체내에서 합성되지 않기 때문에 체내 흡수율이 높아야 한다.
따라서 충분한 양의 비타민을 섭취할 수 있도록 해야 한다.



In [ ]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
text = '피부조직이 괴사된다면'
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)

피부조직이 괴사된다면 그 원인은 아직 밝혀지지 않았다.
하지만 이 같은 증상이 심해지면 혈액순환이 원활하지 못해 혈관이 좁아져 뇌졸중이나 심근경색 등 치명적인 합병증을 유발할 수 있다.
따라서 평소 고혈압, 당뇨병 등의 만성질환을 앓고 있는 사람은 정기적인 검진을 통해 혈관조영술을 받는 것이 좋다.
또한 동맥경화증 치료와 함께 약물치료도 병행해야 한다.
혈관 조영술은 혈관을 넓혀주는 시술로, 수술 후 흉터가 남지 않고 회복기간이 빠르다.
특히 최근에는 레이저를 이용한 미세혈관을 이용해 손상된 조직을 재생시키는 ‘레이저 스캐닝’ 기법이 각광받고 있다.</d> 한국관광공사는


# 번역 사전학습 모델을 가져와서 번역
  - 만약에 학습을 하게 된다면. 번역쌍의 문서로 파인튜닝

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, AdamW
from torch.utils.data import DataLoader
import torch
dataset = load_dataset("PrompTart/PTT_en_ko")

# 모델과 토크나이저 로드
MODEL_NAME = 'skt/kogpt2-base-v2'
tokenizer = GPT2TokenizerFast.from_pretrained(MODEL_NAME)  #Hugging Face 불러옴
# 패딩 토큰 설정
tokenizer.add_special_tokens({'pad_token': '[PAD]'}) # 시퀀스를 동일한 길이로 맞추기 위해서 사용
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME) # 사전학습된 모델로드(전이학습)
model.resize_token_embeddings(len(tokenizer)) # 패딩토큰 추가했으니. 임배딩층 재 조정

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

Embedding(51202, 768)

In [ ]:
# 데이터 전처리 인코딩
input_column = 'english'
output_column = 'korean'

def encode(data):
  inputs = tokenizer(data[input_column], padding='max_length', truncation=True, max_length=512,return_tensors='pt')
  outputs = tokenizer(data[output_column], padding='max_length', truncation=True, max_length=512,return_tensors='pt')
  inputs['labels'] = outputs['input_ids']
  return inputs

In [ ]:
tokenized_dataset = dataset.map(encode, batched=True)

Map:   0%|          | 0/1116 [00:00<?, ? examples/s]

Map:   0%|          | 0/144 [00:00<?, ? examples/s]

Map:   0%|          | 0/138 [00:00<?, ? examples/s]

In [ ]:
# 데이터 로더 정의
train_dataloader = DataLoader(tokenized_dataset['train'], batch_size=8, shuffle=True)
eval_dataloader = DataLoader(tokenized_dataset['validation'], batch_size=8, shuffle=False)
test_dataloader = DataLoader(tokenized_dataset['test'], batch_size=8, shuffle=False)

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# model -> gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51202, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51202, bias=False)
)

In [ ]:
import tensorflow as tf
iterator = tqdm(train_dataloader)
for batch in iterator:
  print(batch['input_ids'])
  break

  0%|          | 0/140 [00:00<?, ?it/s]

[tensor([10553, 10791,  9707,  9707,  9707, 47269, 10791, 14659]), tensor([12153, 13612,   463,   463,   463, 10567,   739,   441]), tensor([  442, 30258, 13799, 13799, 13799,  9776, 14960,  9821]), tensor([ 9714, 20548, 12895, 12895, 12895, 15317, 50091,   454]), tensor([24012, 11571,   445,   445,   445,  9714, 18832,  9821]), tensor([  449, 10421, 38220, 38220, 38220,  9485, 11571, 10272]), tensor([15802, 34957, 10929, 10929, 10929, 33265, 10421, 10929]), tensor([11849,   440, 11780, 11925,   739, 11849, 11780, 11849]), tensor([12768, 14916,   446,  9549, 16363, 12768, 10630, 10845]), tensor([10630, 11060, 24812, 12895, 11899, 10630,   444, 19966]), tensor([  457,  9969,   449,   462, 38832,   457, 45636, 30266]), tensor([  739, 13852, 23112, 31178, 14415, 46359,   459,  9837]), tensor([13271, 14415,   739,  9776,   443, 14778, 19489,   452]), tensor([14257,   443,  9792,   739, 34937, 13612,   460, 10929]), tensor([34957, 34937,  9768, 11899, 14415, 11925,  9969, 38187]), tensor([2

In [ ]:
from tqdm import tqdm
# 파인튜닝 루프
model.train()
for epoch in range(10):
  iterator = tqdm(train_dataloader)
  for batch in iterator:
    optimizer.zero_grad()

    input_ids = torch.stack(batch['input_ids']).to(device)
    attention_mask = torch.stack(batch['attention_mask']).to(device)
    labels = torch.stack(batch['labels']).to(device)

    output = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
    loss = output.loss
    loss.backward()
    optimizer.step()

    iterator.set_description(f"epoch:{epoch+1} Loss: {loss.item():.3f}")

# 파인튜닝 된 모델 저장
model.save_pretrained('en_kr_kogpt2')
tokenizer.save_pretrained('en_kr_kogpt2')

print("모델 파인튜닝이 완료되었습니다.")

epoch:10 Loss: 1.788: 100%|██████████| 140/140 [02:34<00:00,  1.10s/it]


모델 파인튜닝이 완료되었습니다.


In [ ]:
# 번역
test_sentence = 'Adversarial training has emerged as a pivotal mechanism for enhancing the robustness of machine learning models against malicious inputs.'
# 입력 토크나이져
input_ids = tokenizer.encode(test_sentence, return_tensors='pt').to(device)
# 텍스트 생성
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      max_length=512
  )
# 결과 디코딩
translated_sentence = tokenizer.decode(generated_ids[0],skip_special_tokens=True)
print(translated_sentence)

Adversarial training has emerged as a pivotal mechanism for enhancing the robustness of machine learning models against malicious inputs.


In [ ]:
temp = next(iter(train_dataloader))
temp['input_ids'][0], temp['labels'][0]

(tensor([10068,  9707,  9707, 10755, 10755, 27124, 31752, 10791]),
 tensor([48088, 12509,  9803, 12509, 12509,  9024,  9598, 10663]))

In [ ]:
# 옵티마이져 설정


DatasetDict({
    train: Dataset({
        features: ['terms_set_index', 'terms', 'duplicated', 'english', 'korean'],
        num_rows: 1116
    })
    validation: Dataset({
        features: ['terms_set_index', 'terms', 'duplicated', 'english', 'korean'],
        num_rows: 144
    })
    test: Dataset({
        features: ['terms_set_index', 'terms', 'duplicated', 'english', 'korean'],
        num_rows: 138
    })
})

In [ ]:
tokenized_dataset['train'][0]

{'terms_set_index': 0,
 'terms': ['adversarial training',
  'recurrent neural architectures',
  'bayesian optimization'],
 'duplicated': False,
 'english': 'Adversarial training has emerged as a pivotal mechanism for enhancing the robustness of machine learning models against malicious inputs. Recurrent neural architectures have been widely recognized for their capability to effectively capture temporal dependencies in sequential data. Bayesian optimization serves as a powerful strategy for optimizing complex objective functions, particularly in scenarios where evaluations are costly.',
 'korean': '적대적 훈련(adversarial training)은 악성 입력에 대해 머신러닝 모델의 견고성을 향상시키는 데 중추적인 역할을 하게 되었습니다. 순환 신경 구조(recurrent neural architectures)는 시퀀스 데이터에서 시간적 의존성을 효율적으로 포착할 수 있는 능력 때문에 널리 인정받고 있습니다. 베이지안 최적화(bayesian optimization)는 평가 비용이 많이 드는 상황에서 복잡한 목적 함수를 최적화하기 위한 강력한 전략으로 사용됩니다.',
 'input_ids': [27124,
  460,
  14071,
  9837,
  27627,
  10448,
  10631,
  9610,
  10929,
  18125,
  10712,
  739,
  11899,
  9